In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import tarfile 
import scipy.io
import cv2
import os

%matplotlib inline 

Using TensorFlow backend.


In [2]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.neural_network import MLPClassifier

In [3]:
# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']))
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('../data/train')
valid_files, valid_targets = load_dataset('../data/validation')
test_files, test_targets = load_dataset('../data/test')

In [4]:
# return a dictionary that holds all images category by category. 
def load_images_from_folder(folder):
    images = {}
    for filename in os.listdir(folder):
        category = []
        path = folder + "/" + filename
        for dog in os.listdir(path):
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.imread(path + "/" + dog,0)
            if img is not None:
                category.append(img)
        images[filename] = category
    return images

images = load_images_from_folder('../data/train')  # take all images category by category 
test = load_images_from_folder("../data/validation") # take test images 

In [5]:
dico = []
sift = cv2.xfeatures2d.SIFT_create()

def keypoints(folder):
    for filename in os.listdir(folder):
        path = folder + "/" + filename
        for files in os.listdir(path):
            img = cv2.imread(path + "/" + files,0)
            kp, des = sift.detectAndCompute(img, None)
            for d in des:
                dico.append(d)
    return(dico)

In [6]:
kp = keypoints('../data/train')

In [40]:
shape(kp)

NameError: name 'shape' is not defined

In [7]:
def clustering(kp):
    k = 500
    batch_size = 300
    kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size).fit(kp)
    return(kmeans)

In [11]:
clustering(kp)

MiniBatchKMeans(batch_size=300, compute_labels=True, init='k-means++',
                init_size=None, max_iter=100, max_no_improvement=10,
                n_clusters=500, n_init=3, random_state=None,
                reassignment_ratio=0.01, tol=0.0, verbose=0)

In [12]:
def histogram():
    kmeans.verbose = False

    histo_list = []

    for leaf in train.id:
        img = cv2.imread(img_path + str(leaf) + ".jpg")
        kp, des = sift.detectAndCompute(img, None)

        histo = np.zeros(k)
        nkp = np.size(kp)

        for d in des:
            idx = kmeans.predict([d])
            histo[idx] += 1/nkp # normalization
            histo_list.append(histo)

In [13]:
from sklearn import svm

def svm(histo):
    X = np.array(histo)
    y = train_targets
    
    clf = svm.SVC(decision_function_shape='ovo')
    mlp = MLPClassifier(verbose=True, max_iter=600000)
    mlp.fit(X, Y)